In [1]:
import pandas as pd
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()

# Create a DataFrame with the feature data
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

df.to_csv('iris.csv', index=False)

In [2]:
# Add the target variable to the DataFrame
df['target'] = iris.target

# Optionally, display the first few rows
print(df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [3]:
# we do not need to normalize the data because we will implement a decision tree classifier
# which is not sensitive to the scale of the features
# However, we do need to split the data into training and testing sets

from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

# Decision Tree Classifier

Decision tree classifier divides the data into segments to reduce hidden information as much as possible so that when a new datapoint is presented, its class is determined accordingly. 
For it to work, we have two choices:

- Information Gain
- Gini Score

We will go with Information Gain, but implementing the other is no harder.


Let's define entropy first!
Entropy is the amount of hidden information in a state of a system. 


<div align="center">
 
$S = \sum_{i=1}^{n} - p_i \log{p_i}$

</div>

A decision tree has decision nodes that will divide the data into separate parts to "gain" information. That is, when the data fits "a rule" following that rule will give us purer data, with more insights.

So, starting from a root node (with all the data), we start making decisions (dividing the data in subsets), and making sure we do not overfit with either max depth, or min number of datapoints while dividing. 


In [12]:
import numpy as np

class Node:
    def __init__(self, entropy, num_samples, num_samples_per_class, predicted_class):
        self.entropy = entropy
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class  # e.g., {class_label: count}
        self.feature_index = None      # Feature index used for splitting
        self.threshold = None          # Threshold value for the feature split
        self.left = None               # Left child node
        self.right = None              # Right child node
        self.predicted_class = predicted_class  # The majority class (for leaf prediction)

    def is_leaf_node(self):
        return self.left is None and self.right is None


class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        """
        max_depth: Maximum depth of the tree.
        min_samples_split: Minimum samples required to split a node.
        """
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.num_classes = None
        self.num_features = None

    def fit(self, X, y):
        self.num_classes = len(np.unique(y))
        self.num_features = X.shape[1]
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        n_samples = len(y)
        classes, counts = np.unique(y, return_counts=True)
        num_samples_per_class = dict(zip(classes, counts))
        predicted_class = classes[np.argmax(counts)]
        parent_entropy = self._calculate_entropy(y)
        node = Node(
            entropy=parent_entropy,
            num_samples=n_samples,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class
        )

        # Stopping criteria:
        # 1. Maximum depth reached.
        # 2. Not enough samples to split.
        # 3. Node is pure.
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            parent_entropy == 0):
            return node

        # Determine the best split by maximizing information gain.
        best_feature_idx, best_threshold, best_info_gain = self._determine_best_split(X, y, parent_entropy)
        if best_feature_idx is None or best_info_gain <= 0:
            return node  # No valid split

        # Split data on the best feature/threshold.
        left_indices = X[:, best_feature_idx] <= best_threshold
        right_indices = X[:, best_feature_idx] > best_threshold

        # If a split ends up with an empty node, return current node.
        if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
            return node

        node.feature_index = best_feature_idx
        node.threshold = best_threshold
        node.left = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        node.right = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return node

    def _calculate_entropy(self, y):
        """Calculate entropy for labels y."""
        n_samples = len(y)
        if n_samples == 0:
            return 0
        entropy = 0
        classes, counts = np.unique(y, return_counts=True)
        for count in counts:
            p = count / n_samples
            if p > 0:
                entropy -= p * np.log2(p)
        return entropy

    def _determine_best_split(self, X, y, parent_entropy):
        n_samples, n_features = X.shape
        best_info_gain = -float('inf')
        best_feature_idx, best_threshold = None, None

        # Loop over every feature.
        for feature_idx in range(n_features):
            # Sort feature values and corresponding labels.
            sorted_idx = np.argsort(X[:, feature_idx])
            X_sorted = X[sorted_idx, feature_idx]
            y_sorted = y[sorted_idx]

            # Iterate over potential split points (only when the feature value changes).
            for i in range(1, n_samples):
                if X_sorted[i] == X_sorted[i - 1]:
                    continue

                # Use the midpoint between consecutive values as the threshold.
                threshold = (X_sorted[i] + X_sorted[i - 1]) / 2.0

                # Split labels into left and right groups.
                left_y = y_sorted[:i]
                right_y = y_sorted[i:]

                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                # Compute the entropy for the splits.
                left_entropy = self._calculate_entropy(left_y)
                right_entropy = self._calculate_entropy(right_y)

                # Compute weighted average of children entropies.
                weighted_entropy = (len(left_y) / n_samples) * left_entropy + (len(right_y) / n_samples) * right_entropy

                # Information Gain = Parent Entropy - Weighted Entropy of children.
                info_gain = parent_entropy - weighted_entropy

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature_idx = feature_idx
                    best_threshold = threshold

        return best_feature_idx, best_threshold, best_info_gain

    def predict(self, X):
        return np.array([self._predict_instance(x, self.root) for x in X])

    def _predict_instance(self, x, node):
        # If the node is a leaf, return its predicted class.
        if node.is_leaf_node():
            return node.predicted_class

        # Traverse the tree recursively.
        if x[node.feature_index] <= node.threshold:
            return self._predict_instance(x, node.left)
        else:
            return self._predict_instance(x, node.right)


In [19]:
# now we can make some predictions using our own DecisionTree class
import numpy as np
decision_tree = DecisionTree(max_depth=3)
decision_tree.fit(X_train.values, y_train.values)


In [20]:
# now we can get accuracy, recall, precision, and F1 score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 

y_pred = decision_tree.predict(X_test.values)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Precision: {precision:.2f}')
print(f'F1 Score: {f1:.2f}')


Accuracy: 1.00
Recall: 1.00
Precision: 1.00
F1 Score: 1.00


In [9]:
# let's compare our results with the sklearn DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier

sklearn_tree = DecisionTreeClassifier(max_depth=4)
sklearn_tree.fit(X_train, y_train)

y_pred_sklearn = sklearn_tree.predict(X_test)

accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
recall_sklearn = recall_score(y_test, y_pred_sklearn, average='weighted')
precision_sklearn = precision_score(y_test, y_pred_sklearn, average='weighted')
f1_sklearn = f1_score(y_test, y_pred_sklearn, average='weighted')

print(f'Sklearn DecisionTreeClassifier')
print(f'Accuracy: {accuracy_sklearn:.2f}')
print(f'Recall: {recall_sklearn:.2f}')
print(f'Precision: {precision_sklearn:.2f}')
print(f'F1 Score: {f1_sklearn:.2f}')


Sklearn DecisionTreeClassifier
Accuracy: 1.00
Recall: 1.00
Precision: 1.00
F1 Score: 1.00
